In [1]:
import os
import tensorflow as tf
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import multilabel_confusion_matrix, confusion_matrix
import math
from itertools import product
import argparse
import sys
from utils_kmer import *
import calendar
import time

Using TensorFlow backend.


In [2]:
def process_eachseq(kmerfile,mask_seq,new_kmers):
    global kmer3
    if os.path.exists(kmerfile):  #如果pssm文件存在
        #print("found " + kmerfile + "\n")  #输出找到pssm文件+换行
        kmer3 = readKMER(kmerfile)  #读取kmer文件

        

    kmer3 = kmer3.astype(float)  #对kmer的数据类型转换为浮点型
    new_kmers.append(kmer3)  #新的kmer空列表中添加kmer矩阵
    mask_seq.append(gen_mask_mat(1, 0))  #mask序列空列表添加gen_mask矩阵，序列长度为行数，padnum为列数？？？
    
    

In [3]:
def endpad(seqfile, labelfile, kmerdir="", npzfile = ""): #定义endpad(序列文件，标签文件，pssm路径，npz文件)
    if not os.path.exists(npzfile):  #如果npz文件不存在，建立新的pssm空列表，标签空列表，mask序列空列表，id空列表
        new_kmers = []
        labels = []
        mask_seq = []
        ids=[]
        seqs=[]
        f = open(seqfile, "r")  #f为打开序列文件
        f2 = open(labelfile, "r")  #f2为打开标签文件
        line = f.readline()  #读取序列文件的第一行
        while line != '':
            kmerfile = kmerdir + line[1:].strip() + "_kmer.txt"  #kmer文件名=kmer地址+id名+_kmer.txt
            if line[0] == '>':  #如果该行第一个字符为>
                id = line.strip()[1:]  #id为去掉>的字符
                ids.append(id)   #在id空列表中添加id
            label = f2.readline().strip()  #标签为f2（标签文件）中去掉首尾空格的内容
            labels.append(label)  #在标签空列表中添加标签
            seq = f.readline().strip()  #第一次seq为第2行的内容，实际seq为>行的下一行
            #seql = len(seq)   #序列长度  
            process_eachseq(kmerfile,mask_seq,new_kmers)
            line = f.readline()  #继续读取下一行，即>行
        x = np.array(new_kmers)  #把new_pssms列表变为数组，赋给x
        y = [convertlabels_to_categorical(i) for i in labels]  #把标签列表转化为类别(i)
        y = np.array(y)  #再把类别转化为数组
        mask = np.array(mask_seq)  #把mask_seq（标注的序列？）转化为数组
        mask = np.expand_dims(mask, axis=1)
        #mask = np.expand_dims(mask, axis=1)
        np.savez(npzfile, x=x, y=y, mask=mask, ids=ids)  #保存多个数组到同一个文件中,保存格式是.npz
        return [x, y, mask,ids]  #返回pssm矩阵，类别，标注序列，名字id
    else:  #如果上述都存在，直接转化为数组
        mask = np.load(npzfile)['mask']
        x = np.load(npzfile)['x']
        y = np.load(npzfile)['y']
        ids=np.load(npzfile)['ids']
        return [x, y, mask,ids]

In [4]:
def train_MULocDeep(lv1_dir,kmer_dir,output_dir,foldnum):


    # get big data 训练10分类的多分类
    [train_x_big, train_y_big, train_mask_big, train_ids_big] = endpad(
        lv1_dir + "lv1_train_fold" + str(foldnum) + "_seq",
        lv1_dir + "lv1_train_fold" + str(foldnum) + "_lab",
        kmer_dir,
        "E:/论文真的非常重要请好好学习/2021.2 MULocDeep-master/kmer方法的mul/npz文件/lv1_train_fold" + str(foldnum) + "_seq.npz")

    [val_x_big, val_y_big, val_mask_big, val_ids_big] = endpad(
        lv1_dir + "lv1_val_fold" + str(foldnum) + "_seq",
        lv1_dir + "lv1_val_fold" + str(foldnum) + "_lab",
        kmer_dir,
        "E:/论文真的非常重要请好好学习/2021.2 MULocDeep-master/kmer方法的mul/npz文件/lv1_val_fold" + str(foldnum) + "_seq.npz")

    batch_size = 128
    print("doing " + str(foldnum) + "th fold")
    model_big, model_small = singlemodel(train_x_big)  #模型为singlemodel

    filepath_acc_big_lv1 = output_dir+"fold" + str(
        foldnum) + "_big_lv1_acc-weights.hdf5"  # -improvement-{epoch:02d}-{val_loss:.2f}.hdf5"


    filepath_loss_big_lv1 = output_dir+"fold" + str(
        foldnum) + "_big_lv1_loss-weights.hdf5"  # -improvement-{epoch:02d}-{val_loss:.2f}.hdf5"




    checkpoint_acc_big_lev1 = ModelCheckpoint(filepath_acc_big_lv1, monitor='val_accuracy', save_best_only=True,
                                          mode='max',
                                          save_weights_only=True, verbose=1)

    
    checkpoint_loss_big_lev1 = ModelCheckpoint(filepath_loss_big_lv1, monitor='val_loss', save_best_only=True,
                                          mode='min',
                                          save_weights_only=True, verbose=1)
    
   
    
    for i in range(80):
        # train small model
        print("epoch "+str(i)+"\n")
       
        # train big model  
        fitHistory_batch_big = model_big.fit([train_x_big, train_mask_big],
                                             [getTrue4out1(train_y_big)],  #为何大模型没有train_y_big
                                             batch_size=batch_size, epochs=1,  #等于1？？
                                             validation_data=(
                                             [val_x_big, val_mask_big], [getTrue4out1(val_y_big)]),  #也没有val_y_big
                                             callbacks=[checkpoint_acc_big_lev1,checkpoint_loss_big_lev1], verbose=1)


In [5]:
def train_var(input_var,pssm_dir,output_dir,foldnum):
    # get small data
    [train_x,train_y,train_mask,train_ids]=endpad(input_var+"deeploc_40nr_train_fold"+str(foldnum)+"_seq",
                                        input_var+"deeploc_40nr_train_fold"+str(foldnum)+"_label",
                                        pssm_dir,
                                        "D:/deeploc/deeploc_40nr_8folds/train_fold"+str(foldnum)+"_seq.npz")
    [val_x,val_y,val_mask,val_ids]=endpad(input_var+"deeploc_40nr_val_fold"+str(foldnum)+"_seq",
                                  input_var+"deeploc_40nr_val_fold"+str(foldnum)+"_label",
                                  pssm_dir,
                                  "D:/deeploc/deeploc_40nr_8folds/val_fold"+str(foldnum)+"_seq.npz")
    batch_size = 128
    print("doing " + str(foldnum) + "th fold")
    model = var_model(train_x)   #这里的模型是var_model

    filepath_acc = output_dir+"fold" + str(foldnum) + "acc-weights.hdf5"  # -improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
    checkpoint_acc = ModelCheckpoint(filepath_acc, monitor='val_accuracy', save_best_only=True, mode='max',
                                 save_weights_only=True, verbose=1)
    fitHistory_batch = model.fit([train_x,train_mask.reshape(-1,1000,1)],getTrue4out1(train_y),
                                 batch_size=batch_size, epochs=20,
                                 validation_data=([val_x,val_mask.reshape(-1,1000,1)], getTrue4out1(val_y)),
                                 callbacks=[checkpoint_acc],verbose=1)

In [6]:
 '''我们常常可以把argparse的使用简化成下面四个步骤
       1：import argparse
       2：parser = argparse.ArgumentParser()
       3：parser.add_argument()
       4：parser.parse_args()
       上面四个步骤解释如下：首先导入该模块；然后创建一个解析对象；然后向该对象中添加你要关注的命令行参数和选项，
       每一个add_argument方法对应一个你要关注的参数或选项；最后调用parse_args()方法进行解析；解析成功之后即可使用'''
    
def main():
    #加default
    # description= 这个参数简要描述这个程度做什么以及怎么做
    parser=argparse.ArgumentParser(
        description='MULocDeep: interpretable protein localization classifier at sub-cellular and sub-organellar levels')
    #MULocDeep_model  UniLoc-train-20nr
    #--lv1_input_dir/--lv2_input_dir 亚细胞训练数据，包含8折蛋白质序列和标记  需自己添加
    parser.add_argument('--lv1_input_dir', dest='lv1_dir', type=str, 
                        default="E:/论文真的非常重要请好好学习/2021.2 MULocDeep-master/MULocDeep-master/data/UniLoc-train-20nr/8folds/",
                        help='sub-cellular training data, contains 8 folds protein sequences and labels', required=False)

    #--MULocDeep_model 添加它来训练MULocDeep模型，否则训练一个var模型
    parser.add_argument('--MULocDeep_model', dest='modeltype', action='store_false',  #触发，store_true会触发DeepLoc
                        #如果是store_false,则默认值是True，如果是store_true,则默认值是False  
                        help='Add this to train the MULocDeep model, otherwise train a variant model', required=False)
    #--model_output 受过训练的模型存储的目录的名称  需自己添加
    parser.add_argument('--model_output', dest='outputdir', type=str, 
                       default="E:/论文真的非常重要请好好学习/2021.2 MULocDeep-master/kmer方法的mul/result",
                       help='the name of the directory where the trained model stores', required=False)  #由True改成False
    
    parser.add_argument('-existkmer', dest='existkmer', type=str,
                        default="E:/论文真的非常重要请好好学习/2021.2 MULocDeep-master/kmer方法的mul/data/",
                        help='the name of the existing kmer directory if there is one.', required=False)
    
    #var_model  deeploc_40nr_8folds
    #--input_dir 训练var模型的数据，包含8折蛋白质序列和标记  需自己添加
    parser.add_argument('--input_var', dest='var_dir', type=str,
                        default="D:/deeploc/deeploc_40nr_8folds",
                        help='data for traing the variant model, contains 8 folds protein sequences and labels', required=False)
    #改true  并且还需要加一个model_ouput  一个是deeploc  一个是MULocDeep
    parser.add_argument('--var_model_output', dest='var_outputdir', type=str, help='the name of the directory where the trained model stores', 
                        default="D:/deeploc/var_model_result1",
                        required=False)  #由True改成False
    parser.add_argument('-var_existPSSM', dest='var_existPSSM', type=str,
                        default="D:/deeploc/deeploc_pssm",
                        help='the name of the existing PSSM directory if there is one.', required=False)
    parser.set_defaults(feature=True)
    #args = parser.parse_args()   #改
    args = parser.parse_known_args()[0]   #jupyter下运行解析需要此代码
    model_type=args.modeltype
    input_lv1=args.lv1_dir
    outputdir=args.outputdir
    existkmer = args.existkmer


    if model_type==True:
        if not input_lv1[len(input_lv1) - 1] == "/":
            input_lv1 = input_lv1 + "/"
        if not outputdir[len(outputdir) - 1] == "/":
            outputdir = outputdir + "/"
        if not os.path.exists(outputdir):
            os.mkdir(outputdir)
        if existkmer != "":
            if not existkmer[len(existkmer) - 1] == "/":
                existkmer = existkmer + "/"
        if ((existkmer == "") or (not os.path.exists(existkmer))):
            ts = calendar.timegm(time.gmtime())
            kmerdir = outputdir + str(ts) + "_kmer/"
            if not os.path.exists(kmerdir):
                os.makedirs(kmerdir)
            process_input_train(input_lv1 + "lv1_train.txt", kmerdir)
            
            for foldnum in range(8):
                train_MULocDeep(input_lv1, kmerdir, outputdir, foldnum)
        else:
            for foldnum in range(8):
                train_MULocDeep(input_lv1, existkmer, outputdir, foldnum)
    elif model_type==False:
        if not input_var[len(input_var) - 1] == "/":
            input_var = input_var + "/"
        if not var_outputdir[len(var_outputdir) - 1] == "/":
            var_outputdir = var_outputdir + "/"
        if not os.path.exists(var_outputdir):
            os.mkdir(var_outputdir)
        if existPSSM != "":
            if not var_existPSSM[len(var_existPSSM) - 1] == "/":
                var_existPSSM = var_existPSSM + "/"
        if ((var_existPSSM == "") or (not os.path.exists(var_existPSSM))):
            ts = calendar.timegm(time.gmtime())
            pssmdir = var_outputdir + str(ts) + "_pssm/"
            if not os.path.exists(pssmdir):
                os.makedirs(pssmdir)
            process_input_train(input_var + "processed_deeploc_train_S_seq", pssmdir)
            for foldnum in range(8):
                train_var(input_var, pssmdir, var_outputdir, foldnum)
        else:
            for foldnum in range(8):
                train_var(input_var, var_existPSSM, var_outputdir, foldnum)

In [7]:
if __name__ == "__main__":
    main()

doing 0th fold
Instructions for updating:
Colocations handled automatically by placer.
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              (None, 1, 100)       0                                            
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 1, 100)       0           Input[0][0]                      
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 1, 1)         0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 1, 100)       0           dropout_1[0][0]                  
     

Train on 17138 samples, validate on 2445 samples
Epoch 1/1
17138/17138 [==============================] - 7s 388us/step - loss: 1.9949 - accuracy: 0.4753 - val_loss: 1.8857 - val_accuracy: 0.5535

Epoch 00001: val_accuracy improved from -inf to 0.55346, saving model to E:/论文真的非常重要请好好学习/2021.2 MULocDeep-master/kmer方法的mul/result/fold0_big_lv1_acc-weights.hdf5

Epoch 00001: val_loss improved from inf to 1.88572, saving model to E:/论文真的非常重要请好好学习/2021.2 MULocDeep-master/kmer方法的mul/result/fold0_big_lv1_loss-weights.hdf5
epoch 1

Train on 17138 samples, validate on 2445 samples
Epoch 1/1
17138/17138 [==============================] - 4s 207us/step - loss: 1.8727 - accuracy: 0.7284 - val_loss: 1.8515 - val_accuracy: 0.8420

Epoch 00001: val_accuracy improved from 0.55346 to 0.84196, saving model to E:/论文真的非常重要请好好学习/2021.2 MULocDeep-master/kmer方法的mul/result/fold0_big_lv1_acc-weights.hdf5

Epoch 00001: val_loss improved from 1.88572 to 1.85153, saving model to E:/论文真的非常重要请好好学习/2021.2 MULocDeep-m

17138/17138 [==============================] - 4s 240us/step - loss: 1.5458 - accuracy: 0.8854 - val_loss: 1.5826 - val_accuracy: 0.8617

Epoch 00001: val_accuracy improved from 0.85166 to 0.86172, saving model to E:/论文真的非常重要请好好学习/2021.2 MULocDeep-master/kmer方法的mul/result/fold0_big_lv1_acc-weights.hdf5

Epoch 00001: val_loss improved from 1.59112 to 1.58259, saving model to E:/论文真的非常重要请好好学习/2021.2 MULocDeep-master/kmer方法的mul/result/fold0_big_lv1_loss-weights.hdf5
epoch 19

Train on 17138 samples, validate on 2445 samples
Epoch 1/1
17138/17138 [==============================] - 4s 232us/step - loss: 1.5369 - accuracy: 0.8881 - val_loss: 1.5763 - val_accuracy: 0.8606

Epoch 00001: val_accuracy did not improve from 0.86172

Epoch 00001: val_loss improved from 1.58259 to 1.57627, saving model to E:/论文真的非常重要请好好学习/2021.2 MULocDeep-master/kmer方法的mul/result/fold0_big_lv1_loss-weights.hdf5
epoch 20

Train on 17138 samples, validate on 2445 samples
Epoch 1/1
17138/17138 [========================

17138/17138 [==============================] - 4s 233us/step - loss: 1.4432 - accuracy: 0.9043 - val_loss: 1.5208 - val_accuracy: 0.8641

Epoch 00001: val_accuracy did not improve from 0.86822

Epoch 00001: val_loss improved from 1.52561 to 1.52084, saving model to E:/论文真的非常重要请好好学习/2021.2 MULocDeep-master/kmer方法的mul/result/fold0_big_lv1_loss-weights.hdf5
epoch 38

Train on 17138 samples, validate on 2445 samples
Epoch 1/1
17138/17138 [==============================] - 4s 234us/step - loss: 1.4408 - accuracy: 0.9039 - val_loss: 1.5259 - val_accuracy: 0.8695

Epoch 00001: val_accuracy improved from 0.86822 to 0.86949, saving model to E:/论文真的非常重要请好好学习/2021.2 MULocDeep-master/kmer方法的mul/result/fold0_big_lv1_acc-weights.hdf5

Epoch 00001: val_loss did not improve from 1.52084
epoch 39

Train on 17138 samples, validate on 2445 samples
Epoch 1/1
17138/17138 [==============================] - 4s 232us/step - loss: 1.4376 - accuracy: 0.9049 - val_loss: 1.5218 - val_accuracy: 0.8681

Epoch 00001

17138/17138 [==============================] - 4s 258us/step - loss: 1.4037 - accuracy: 0.9086 - val_loss: 1.5232 - val_accuracy: 0.8685

Epoch 00001: val_accuracy did not improve from 0.87194

Epoch 00001: val_loss did not improve from 1.51637
epoch 62

Train on 17138 samples, validate on 2445 samples
Epoch 1/1
17138/17138 [==============================] - 4s 246us/step - loss: 1.4037 - accuracy: 0.9081 - val_loss: 1.5233 - val_accuracy: 0.8656

Epoch 00001: val_accuracy did not improve from 0.87194

Epoch 00001: val_loss did not improve from 1.51637
epoch 63

Train on 17138 samples, validate on 2445 samples
Epoch 1/1
17138/17138 [==============================] - 4s 243us/step - loss: 1.4020 - accuracy: 0.9087 - val_loss: 1.5228 - val_accuracy: 0.8641

Epoch 00001: val_accuracy did not improve from 0.87194

Epoch 00001: val_loss did not improve from 1.51637
epoch 64

Train on 17138 samples, validate on 2445 samples
Epoch 1/1
17138/17138 [==============================] - 4s 248us/st

...

17077/17077 [==============================] - 5s 267us/step - loss: 1.4056 - accuracy: 0.9084 - val_loss: 1.5122 - val_accuracy: 0.8709

Epoch 00001: val_accuracy did not improve from 0.87522

Epoch 00001: val_loss did not improve from 1.50534
epoch 57

Train on 17077 samples, validate on 2506 samples
Epoch 1/1
17077/17077 [==============================] - 4s 255us/step - loss: 1.4062 - accuracy: 0.9084 - val_loss: 1.5096 - val_accuracy: 0.8722

Epoch 00001: val_accuracy did not improve from 0.87522

Epoch 00001: val_loss did not improve from 1.50534
epoch 58

Train on 17077 samples, validate on 2506 samples
Epoch 1/1
17077/17077 [==============================] - 4s 249us/step - loss: 1.4041 - accuracy: 0.9088 - val_loss: 1.5143 - val_accuracy: 0.8690

Epoch 00001: val_accuracy did not improve from 0.87522

Epoch 00001: val_loss did not improve from 1.50534
epoch 59

Train on 17077 samples, validate on 2506 samples
Epoch 1/1
17077/17077 [==============================] - 4s 251us/st

Train on 17053 samples, validate on 2530 samples
Epoch 1/1
17053/17053 [==============================] - 8s 476us/step - loss: 2.0143 - accuracy: 0.4635 - val_loss: 1.9079 - val_accuracy: 0.5617

Epoch 00001: val_accuracy improved from -inf to 0.56170, saving model to E:/论文真的非常重要请好好学习/2021.2 MULocDeep-master/kmer方法的mul/result/fold7_big_lv1_acc-weights.hdf5

Epoch 00001: val_loss improved from inf to 1.90794, saving model to E:/论文真的非常重要请好好学习/2021.2 MULocDeep-master/kmer方法的mul/result/fold7_big_lv1_loss-weights.hdf5
epoch 1

Train on 17053 samples, validate on 2530 samples
Epoch 1/1
17053/17053 [==============================] - 4s 239us/step - loss: 1.9056 - accuracy: 0.6036 - val_loss: 1.8411 - val_accuracy: 0.7383

Epoch 00001: val_accuracy improved from 0.56170 to 0.73834, saving model to E:/论文真的非常重要请好好学习/2021.2 MULocDeep-master/kmer方法的mul/result/fold7_big_lv1_acc-weights.hdf5

Epoch 00001: val_loss improved from 1.90794 to 1.84107, saving model to E:/论文真的非常重要请好好学习/2021.2 MULocDeep-m

17053/17053 [==============================] - 4s 243us/step - loss: 1.5551 - accuracy: 0.8830 - val_loss: 1.5766 - val_accuracy: 0.8606

Epoch 00001: val_accuracy improved from 0.85470 to 0.86063, saving model to E:/论文真的非常重要请好好学习/2021.2 MULocDeep-master/kmer方法的mul/result/fold7_big_lv1_acc-weights.hdf5

Epoch 00001: val_loss improved from 1.58432 to 1.57662, saving model to E:/论文真的非常重要请好好学习/2021.2 MULocDeep-master/kmer方法的mul/result/fold7_big_lv1_loss-weights.hdf5
epoch 18

Train on 17053 samples, validate on 2530 samples
Epoch 1/1
17053/17053 [==============================] - 4s 243us/step - loss: 1.5456 - accuracy: 0.8867 - val_loss: 1.5701 - val_accuracy: 0.8634

Epoch 00001: val_accuracy improved from 0.86063 to 0.86336, saving model to E:/论文真的非常重要请好好学习/2021.2 MULocDeep-master/kmer方法的mul/result/fold7_big_lv1_acc-weights.hdf5

Epoch 00001: val_loss improved from 1.57662 to 1.57011, saving model to E:/论文真的非常重要请好好学习/2021.2 MULocDeep-master/kmer方法的mul/result/fold7_big_lv1_loss-weights.

17053/17053 [==============================] - 4s 245us/step - loss: 1.4526 - accuracy: 0.9031 - val_loss: 1.5083 - val_accuracy: 0.8730

Epoch 00001: val_accuracy did not improve from 0.87494

Epoch 00001: val_loss improved from 1.50929 to 1.50827, saving model to E:/论文真的非常重要请好好学习/2021.2 MULocDeep-master/kmer方法的mul/result/fold7_big_lv1_loss-weights.hdf5
epoch 35

Train on 17053 samples, validate on 2530 samples
Epoch 1/1
17053/17053 [==============================] - 5s 265us/step - loss: 1.4488 - accuracy: 0.9036 - val_loss: 1.5075 - val_accuracy: 0.8722

Epoch 00001: val_accuracy did not improve from 0.87494

Epoch 00001: val_loss improved from 1.50827 to 1.50752, saving model to E:/论文真的非常重要请好好学习/2021.2 MULocDeep-master/kmer方法的mul/result/fold7_big_lv1_loss-weights.hdf5
epoch 36

Train on 17053 samples, validate on 2530 samples
Epoch 1/1
17053/17053 [==============================] - 5s 275us/step - loss: 1.4461 - accuracy: 0.9041 - val_loss: 1.5034 - val_accuracy: 0.8741

Epoch 0000

17053/17053 [==============================] - 4s 245us/step - loss: 1.4104 - accuracy: 0.9076 - val_loss: 1.4991 - val_accuracy: 0.8725

Epoch 00001: val_accuracy did not improve from 0.87561

Epoch 00001: val_loss did not improve from 1.49295
epoch 56

Train on 17053 samples, validate on 2530 samples
Epoch 1/1
17053/17053 [==============================] - 4s 247us/step - loss: 1.4087 - accuracy: 0.9083 - val_loss: 1.4973 - val_accuracy: 0.8728

Epoch 00001: val_accuracy did not improve from 0.87561

Epoch 00001: val_loss did not improve from 1.49295
epoch 57

Train on 17053 samples, validate on 2530 samples
Epoch 1/1
17053/17053 [==============================] - 4s 246us/step - loss: 1.4076 - accuracy: 0.9081 - val_loss: 1.4976 - val_accuracy: 0.8723

Epoch 00001: val_accuracy did not improve from 0.87561

Epoch 00001: val_loss did not improve from 1.49295
epoch 58

Train on 17053 samples, validate on 2530 samples
Epoch 1/1
17053/17053 [==============================] - 4s 257us/st